In [1]:
from openai import OpenAI
import pandas as pd
client = OpenAI()

df = pd.read_csv('sentences.csv')

def translate(arb):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an expert in Moroccan Darija dialect and in translating Darija to English." +
             "Purpose of the translation: To study the accuracy of gpt api on Darija translation to English."
             "Target audience: Darija Computational linguists."
            },
            {"role": "user", "content": "Translate the provided Moroccan Darija dialect sentence into English using word-by-word translation."             },
            {"role": "assistant", "content": "OK, I understand. I am ready to translate to English."},
            {"role": "user", "content": "هوما مخبّيين شي حاجة, أنا متيقّن!"},
            {"role": "assistant", "content": "They're hiding something, I'm sure!"},
            {"role": "user", "content":  f"{arb}" }
            ], 
            #
        temperature=0.1  # Deterministic responses
        )
    return completion.choices[0].message.content
#df['transl'] = df['darija_ar'].apply(translate)
sentence = translate(df['darija_ar'][1])

# Display or save the resulting DataFrame
print(sentence)

Seems like they are trying to stay cool.


First model: comet-ml and paraphrase-MiniLM-L6-v2 suggetion

In [10]:
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained SentenceTransformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Define the two sentences
s1 = "Seems like they are trying to stay cool."
s2 = "It's obvious they're trying to keep their cool."

w1 = "we leave"
w2 = "to go out"

# Encode the sentences to get their embeddings
semb1 = model.encode(s1, convert_to_tensor=True)
semb2 = model.encode(s2, convert_to_tensor=True)

wemb1 = model.encode(w1, convert_to_tensor=True)
wemb2 = model.encode(w2, convert_to_tensor=True)


# Calculate the cosine similarity between the embeddings
s_similarity = util.pytorch_cos_sim(semb1, semb2)
w_similarity = util.pytorch_cos_sim(wemb1, wemb2)

print(f"sentece similarity: {s_similarity.item()}")
print(f"verb similarity: {w_similarity.item()}")


c:\Users\yassi\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


sentece similarity: 0.7992310523986816
verb similarity: 0.48648974299430847


second model: Bert based transformer

In [11]:
from sentence_transformers import SentenceTransformer, util
#from sklearn.metrics.pariwise import cosine_similarity

model_name = 'bert-base-nli-mean-tokens'
# Load a pre-trained SentenceTransformer model
model = SentenceTransformer(model_name)

# Define the two sentences
sentence1 = sentence
sentence2 = df['eng'][1]

# Encode the sentences to get their embeddings
embedding1 = model.encode(sentence1, convert_to_tensor=True)
embedding2 = model.encode(sentence2, convert_to_tensor=True)

# Calculate the cosine similarity between the embeddings
similarity = util.pytorch_cos_sim(embedding1, embedding2)

print(sentence1)
print(sentence2)
print(f"Similarity: {similarity.item()}")

Seems like they are trying to stay cool.
It's obvious they're trying to keep their cool.
Similarity: 0.9145830869674683


In [12]:
word1 = "we leave"
word2 = "to go out"

# Encode the sentences to get their embeddings
embedding1 = model.encode(word1, convert_to_tensor=True)
embedding2 = model.encode(word2, convert_to_tensor=True)

# Calculate the cosine similarity between the embeddings
similarity = util.pytorch_cos_sim(embedding1, embedding2)

print(word1)
print(word2)
print(f"Similarity: {similarity.item()}")

we leave
to go out
Similarity: 0.8262754678726196


Results:

first model:
it doesnt perform well on singular words

second model:
performs better on sentences than the first and on words as well.
# ------------------

##### This is the same metric using a different library sikit learn

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
sentences = ["we leave", "to go out"]
sentence_vecs = model.encode(sentences)
sentence_vecs.shape
cosine_similarity([sentence_vecs[0]],[sentence_vecs[1]])

array([[0.8262754]], dtype=float32)